In [124]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, Document, VectorStoreIndex, StorageContext, load_index_from_storage, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, FunctionTool, ToolMetadata


In [151]:
llm = Ollama(model="llama3.1", request_timeout=120.0, base_url="https://workstation.tail5ac87b.ts.net", verbose=True)
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

Settings.llm = llm
Settings.embed_model = embed_model

In [152]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [153]:
PERSIST_DIR = "lyrics_store"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [154]:
lyrics_query_engine = index.as_query_engine()
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Notes:
    Whenever the user asks for a specific song lyrics, only retrieve the lyrics with no tone/sentiment added.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)
lyrics_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [155]:
# print(lyrics_query_engine.query("cual es una cancion alegra de los kjarkas?"))

In [156]:
import wikipediaapi

def get_artist_wikipedia_info(artist: str) -> str:
    """
        Useful for getting artist information from wikipedia
        Use plain text as input to the tool, this should be the name of an artist or a band.
        The output of this tool is the summary of the wikipedia page corresponding to the artist.
    """
    wiki = wikipediaapi.Wikipedia("Test Wikipedia (eduardo.laruta@gmail.com)", "es")
    page = wiki.page(artist)
    if not page.exists():
        return f"Wikipedia page for {artist} does not exist"
    # print("====")
    # print(page.sections[0])
    # print("====")
    return f"{page.summary}\n{page.sections[0]}\n{page.sections[1]}"

print(get_artist_wikipedia_info("Kjarkas"))


Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio Los Kjarkas surgió como una necesidad económica para sus integrantes, yendo de pueblo en pueblo tocando zamba argentina, pues la música autóctona de Bolivia aun no tenía demasiada repercusión. Sin embargo, poco a poco, diversas peñas comenzaron a hacer énfasis en ritmos más propios del país c

In [157]:
lyrics_description = """
A set of lyrics for songs from the Bolivian Folk Group Los Kjarkas. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""

tools = [
    QueryEngineTool(
        query_engine=lyrics_query_engine,
        metadata=ToolMetadata(
            name="Kjarkas_songs_lyrics",
            description=lyrics_description,
            return_direct=False
        )
    ),
    FunctionTool.from_defaults(get_artist_wikipedia_info)
]
agent = ReActAgent.from_tools(tools, verbose=True)

In [158]:
print(agent.chat("quienes son los kjarkas?"))

> Running step 6cfc0722-036d-4fdc-bef1-8b6e8573f61e. Step input: quienes son los kjarkas?
Thought: The current language of the user is: spanish. I need to use a tool to help me answer the question.
Action: get_artist_wikipedia_info
Action Input: {'artist': 'Los Kjarkas'}
Observation: Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio Los Kjar

In [159]:
print(agent.chat("cual es su canción mas conocida?"))

> Running step 706d77af-d823-46cc-978e-b16bef0a6466. Step input: cual es su canción mas conocida?
Thought: The current language of the user is Spanish. I need to use a tool to help me answer the question.
Action: get_artist_wikipedia_info
Action Input: {'artist': 'Los Kjarkas'}
Observation: Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio L

In [160]:
print(agent.chat("dame la letra"))

> Running step 9ee5b038-e24f-48ce-885d-96722c43fa29. Step input: dame la letra
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: Kjarkas_songs_lyrics
Action Input: {'input': 'Bolivia'}
Observation: **Bolivia de Kjarkas**

"Ser tu brabura / ser la fuerza y juventud / de tu letrado mudo la voz / de inquietud."

* El tema principal del canto es la celebración y el orgullo por la cultura boliviana.
* La canción tiene un tono muy optimista y alegre, como se puede apreciar en la primera estrofa.
	+ Tono/Sentido: Optimismo y Orgullo cultural
* El título "Bolivia" es una expresión de amor y admiración por el país y su gente.

**Fuente:** Kjarkas - Bolivia (álbum no especificado)

Espero que esta información sea útil. ¿Hay algo más en lo que pueda ayudarte?
> Running step a95f343d-23be-4485-abf2-630788eb232e. Step input: None
Thought: The user provided the lyrics and a brief summary of the song "Bolivia" by Los Kjarkas, but I nee

ValueError: Reached max iterations.